# BERT and Path signature

In [ ]:
# solve issue with autocomplete
%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import torch
import pickle
import re

# import sys
# sys.path.insert(0, "../../timeline_generation/")  # Adds higher directory to python modules path
# import src.data_handler

## Load dataset

In [ ]:
# loadHF: load a dataset from hugging face
from src import loadHF
data_loader = loadHF(dataset_name="newspop", 
                     split_name="train")

In [ ]:
# Note: default_preproces_newspop is implemented in loadHF
data_loader.load_preprocessed_df(default_preprocess="newspop")

In [ ]:
dataset_df = data_loader.dataset_df

In [ ]:
dataset_df.head()

## Encode labels

In [ ]:
dataset_df = dataset_df[:500]
dataset_df["label"].value_counts()

In [ ]:
model_specifics = {
    "global_embedding_tp": 'SBERT', #options: SBERT, BERT_cls , BERT_mean, BERT_max
    "dimensionality_reduction_tp": 'ppapca', #options: ppapca, ppapcappa, umap
    "dimensionality_reduction_components": 10, # options: any int number between 1 and embedding dimensions
    "time_injection_history_tp": 'timestamp', #options: timestamp, None
    "time_injection_post_tp": 'timestamp', #options: timestamp, timediff, None
    "signature_dimensions": 3, #options: any int number larger than 1
    "post_embedding_tp": 'sentence', #options: sentence, reduced
    "feature_combination_method": 'attention', #options concatenation, attention 
    "signature_tp": 'log', # options: log, sig
    "classifier_name": 'FFN2hidden', # options: FFN2hidden (any future classifiers added)
    "classes_num": '3class', #options: 3class (5class to be added in the future)
}

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = dataset_df["content"].to_list()

In [ ]:
sentences[:10]

In [ ]:
st_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
embeddings_sentence = st_model.encode(sentences,
                                      batch_size=64,
                                      show_progress_bar=True,
                                      output_value='sentence_embedding', 
                                      convert_to_numpy=True,
                                      convert_to_tensor=False,
                                      device=None,
                                      normalize_embeddings=False,
                                     )

In [ ]:
#dimensionality reduction
from src import DimensionalityReduction

reduction = DimensionalityReduction(method= model_specifics['dimensionality_reduction_tp'], 
                                    components=model_specifics['dimensionality_reduction_components'])

In [ ]:
embeddings_reduced = reduction.fit_transform(embeddings_sentence)

In [ ]:
print(embeddings_sentence.shape)
print(embeddings_reduced.shape)

In [ ]:
embeddings_reduced

In [ ]:
#concatenate new dataframe
from src.dataset import get_modeling_dataframe
df = get_modeling_dataframe(dataset_df, embeddings_sentence, embeddings_reduced)

#get time features
from src.timeinjection import TimeFeatures, Padding
tf = TimeFeatures()
df = tf.get_time_features(df)


#padding
pad = Padding()
df_padded = pad.pad_timelines(df)
df_padded.shape

In [ ]:
if (model_specifics['time_injection_history_tp'] == 'timestamp'):
    path = torch.from_numpy(df_padded[: , : , 2:].astype(float))
else:
    path = torch.from_numpy(df_padded[: , : , 3:].astype(float))

if (model_specifics['time_injection_post_tp']== 'timestamp'):
    time_feature = torch.tensor((df[['time_encoding']].values - df['time_encoding'].mean()) / df['time_encoding'].std() )
    post_time = True
elif (model_specifics['time_injection_post_tp']== 'timediff'):
    time_feature = torch.tensor( (df[['time_diff']].values - df['time_diff'].mean()) / df['time_diff'].std()  )
    post_time = True  
else: 
    time_feature = None
    post_time = False

if (model_specifics['post_embedding_tp'] == 'sentence'):
    bert_embeddings = torch.tensor(df[[c for c in df.columns if re.match("^e\w*[0-9]", c)]].values)
else:
    bert_embeddings = None

In [ ]:
#path = path.squeeze()

In [ ]:
#calculate paths
from src.dyadic_path import DyadicSignatures

dsig = DyadicSignatures(original_size = df.shape[0], 
                        d = path.shape[2], 
                        sig_d = model_specifics['signature_dimensions'],
                        intervals = 1/12, 
                        k_history= None, 
                        embedding_tp = model_specifics['post_embedding_tp'],
                        method = model_specifics['feature_combination_method'],
                        history_tp = model_specifics['signature_tp'], 
                        add_time = post_time)

In [ ]:
sig, last_index_dt_all = dsig.compute_signatures(path)
sig_combined = dsig.combine_signatures(sig)

In [ ]:
x_data = dsig.create_features(path, sig_combined, last_index_dt_all, bert_embeddings, time_feature)

In [ ]:
sig.shape, last_index_dt_all.shape, sig_combined.shape, x_data.shape